<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# Final Project: Data Analysis using Spark

Estimated time needed: **60** minutes

This final project is similar to the Practice Project you did. In this project, you will not be provided with hints or solutions. You will create a DataFrame by loading data from a CSV file and apply transformations and actions using Spark SQL. This needs to be achieved by performing the following tasks:

- Task 1: Generate DataFrame from CSV data.
- Task 2: Define a schema for the data.
- Task 3: Display schema of DataFrame.
- Task 4: Create a temporary view.
- Task 5: Execute an SQL query.
- Task 6: Calculate Average Salary by Department.
- Task 7: Filter and Display IT Department Employees.
- Task 8: Add 10% Bonus to Salaries.
- Task 9: Find Maximum Salary by Age.
- Task 10: Self-Join on Employee Data.
- Task 11: Calculate Average Employee Age.
- Task 12: Calculate Total Salary by Department.
- Task 13: Sort Data by Age and Salary.
- Task 14: Count Employees in Each Department.
- Task 15: Filter Employees with the letter o in the Name.


### Prerequisites 

1. For this lab assignment, you will be using Python and Spark (PySpark). Therefore, it's essential to make sure that the following libraries are installed in your lab environment or within Skills Network (SN) Labs


In [1]:
# Installing required packages  

!pip install pyspark  findspark wget


  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9656 sha256=4281bcc9bd84f231903f97b9749810a4e4518003f7bda540c30383ab42a840d8
  Stored in directory: /home/thangquang/.cache/pip/wheels/01/46/3b/e29ffbe4ebe614ff224bad40fc6a5773a67a163251585a13a9
Successfully built wget


In [1]:
import findspark

findspark.init()

In [2]:
# PySpark is the Spark API for Python. In this lab, we use PySpark to initialize the SparkContext.   

from pyspark import SparkContext, SparkConf

from pyspark.sql import SparkSession

In [3]:
# Creating a SparkContext object  

sc = SparkContext.getOrCreate()

# Creating a SparkSession  

spark = SparkSession \
    .builder \
    .appName("Python Spark DataFrames basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

24/10/30 10:32:24 WARN Utils: Your hostname, thangquang-Aspire-A715-41G resolves to a loopback address: 127.0.1.1; using 172.19.200.233 instead (on interface wlp4s0)
24/10/30 10:32:24 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/30 10:32:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


2. Download the CSV data.  


In [5]:
# Download the CSV data first into a local `employees.csv` file
import wget
wget.download("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-BD0225EN-SkillsNetwork/data/employees.csv")

'employees.csv'

### Tasks


#### Task 1: Generate a Spark DataFrame from the CSV data

Read data from the provided CSV file, `employees.csv` and import it into a Spark DataFrame variable named `employees_df`.

 


In [5]:
# Read data from the "emp" CSV file and import it into a DataFrame variable named "employees_df"  
employees_df = spark.read.csv("employees.csv", header=True, inferSchema=True)

In [6]:
employees_df.printSchema()

root
 |-- Emp_No: integer (nullable = true)
 |-- Emp_Name: string (nullable = true)
 |-- Salary: integer (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Department: string (nullable = true)



#### Task 2: Define a schema for the data

Construct a schema for the input data and then utilize the defined schema to read the CSV file to create a DataFrame named `employees_df`.  


In [10]:
# Define a Schema for the input data and read the file using the user-defined Schema
from pyspark.sql.types import StructField, StringType, IntegerType, StructType

schema = StructType([
    StructField("Emp_No", IntegerType(), True),
    StructField("Emp_Name", StringType(), True),
    StructField("Salary", IntegerType(), True),
    StructField("Age", IntegerType(), True),
    StructField("Department", StringType(), True),
])

employees_df = spark.read.csv("employees.csv", header=True, schema=schema)
employees_df.show()

+------+---------+------+---+----------+
|Emp_No| Emp_Name|Salary|Age|Department|
+------+---------+------+---+----------+
|   198|   Donald|  2600| 29|        IT|
|   199|  Douglas|  2600| 34|     Sales|
|   200| Jennifer|  4400| 36| Marketing|
|   201|  Michael| 13000| 32|        IT|
|   202|      Pat|  6000| 39|        HR|
|   203|    Susan|  6500| 36| Marketing|
|   204|  Hermann| 10000| 29|   Finance|
|   205|  Shelley| 12008| 33|   Finance|
|   206|  William|  8300| 37|        IT|
|   100|   Steven| 24000| 39|        IT|
|   101|    Neena| 17000| 27|     Sales|
|   102|      Lex| 17000| 37| Marketing|
|   103|Alexander|  9000| 39| Marketing|
|   104|    Bruce|  6000| 38|        IT|
|   105|    David|  4800| 39|        IT|
|   106|    Valli|  4800| 38|     Sales|
|   107|    Diana|  4200| 35|     Sales|
|   108|    Nancy| 12008| 28|     Sales|
|   109|   Daniel|  9000| 35|        HR|
|   110|     John|  8200| 31| Marketing|
+------+---------+------+---+----------+
only showing top

#### Task 3: Display schema of DataFrame

Display the schema of the `employees_df` DataFrame, showing all columns and their respective data types.  


In [11]:
# Display all columns of the DataFrame, along with their respective data types
employees_df.printSchema()

root
 |-- Emp_No: integer (nullable = true)
 |-- Emp_Name: string (nullable = true)
 |-- Salary: integer (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Department: string (nullable = true)



#### Task 4: Create a temporary view

Create a temporary view named `employees` for the `employees_df` DataFrame, enabling Spark SQL queries on the data. 


In [12]:
# Create a temporary view named "employees" for the DataFrame
employees_df.createTempView("employees")

result = spark.sql("SELECT * FROM employees")
result.show()

+------+---------+------+---+----------+
|Emp_No| Emp_Name|Salary|Age|Department|
+------+---------+------+---+----------+
|   198|   Donald|  2600| 29|        IT|
|   199|  Douglas|  2600| 34|     Sales|
|   200| Jennifer|  4400| 36| Marketing|
|   201|  Michael| 13000| 32|        IT|
|   202|      Pat|  6000| 39|        HR|
|   203|    Susan|  6500| 36| Marketing|
|   204|  Hermann| 10000| 29|   Finance|
|   205|  Shelley| 12008| 33|   Finance|
|   206|  William|  8300| 37|        IT|
|   100|   Steven| 24000| 39|        IT|
|   101|    Neena| 17000| 27|     Sales|
|   102|      Lex| 17000| 37| Marketing|
|   103|Alexander|  9000| 39| Marketing|
|   104|    Bruce|  6000| 38|        IT|
|   105|    David|  4800| 39|        IT|
|   106|    Valli|  4800| 38|     Sales|
|   107|    Diana|  4200| 35|     Sales|
|   108|    Nancy| 12008| 28|     Sales|
|   109|   Daniel|  9000| 35|        HR|
|   110|     John|  8200| 31| Marketing|
+------+---------+------+---+----------+
only showing top

#### Task 5: Execute an SQL query

Compose and execute an SQL query to fetch the records from the `employees` view where the age of employees exceeds 30. Then, display the result of the SQL query, showcasing the filtered records.


In [13]:
# SQL query to fetch solely the records from the View where the age exceeds 30
result = spark.sql("SELECT * FROM employees WHERE age > 30")
result.show()

+------+-----------+------+---+----------+
|Emp_No|   Emp_Name|Salary|Age|Department|
+------+-----------+------+---+----------+
|   199|    Douglas|  2600| 34|     Sales|
|   200|   Jennifer|  4400| 36| Marketing|
|   201|    Michael| 13000| 32|        IT|
|   202|        Pat|  6000| 39|        HR|
|   203|      Susan|  6500| 36| Marketing|
|   205|    Shelley| 12008| 33|   Finance|
|   206|    William|  8300| 37|        IT|
|   100|     Steven| 24000| 39|        IT|
|   102|        Lex| 17000| 37| Marketing|
|   103|  Alexander|  9000| 39| Marketing|
|   104|      Bruce|  6000| 38|        IT|
|   105|      David|  4800| 39|        IT|
|   106|      Valli|  4800| 38|     Sales|
|   107|      Diana|  4200| 35|     Sales|
|   109|     Daniel|  9000| 35|        HR|
|   110|       John|  8200| 31| Marketing|
|   111|     Ismael|  7700| 32|        IT|
|   112|Jose Manuel|  7800| 34|        HR|
|   113|       Luis|  6900| 34|     Sales|
|   116|     Shelli|  2900| 37|   Finance|
+------+---

#### Task 6: Calculate Average Salary by Department

Compose an SQL query to retrieve the average salary of employees grouped by department. Display the result.


In [14]:
employees_df.printSchema()

root
 |-- Emp_No: integer (nullable = true)
 |-- Emp_Name: string (nullable = true)
 |-- Salary: integer (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Department: string (nullable = true)



In [15]:
# SQL query to calculate the average salary of employees grouped by department
result = spark.sql("SELECT Department, AVG(Salary) as AVG_Salary FROM employees GROUP BY Department")
result.show()

+----------+-----------------+
|Department|       AVG_Salary|
+----------+-----------------+
|     Sales|5492.923076923077|
|        HR|           5837.5|
|   Finance|           5730.8|
| Marketing|6633.333333333333|
|        IT|           7400.0|
+----------+-----------------+



#### Task 7: Filter and Display IT Department Employees

Apply a filter on the `employees_df` DataFrame to select records where the department is `'IT'`. Display the filtered DataFrame.


In [17]:
# Apply a filter to select records where the department is 'IT'
from pyspark.sql.functions import col
filtered_df = employees_df.filter(col("Department") == "IT")
filtered_df.show()

+------+--------+------+---+----------+
|Emp_No|Emp_Name|Salary|Age|Department|
+------+--------+------+---+----------+
|   198|  Donald|  2600| 29|        IT|
|   201| Michael| 13000| 32|        IT|
|   206| William|  8300| 37|        IT|
|   100|  Steven| 24000| 39|        IT|
|   104|   Bruce|  6000| 38|        IT|
|   105|   David|  4800| 39|        IT|
|   111|  Ismael|  7700| 32|        IT|
|   129|   Laura|  3300| 38|        IT|
|   132|      TJ|  2100| 34|        IT|
|   136|   Hazel|  2200| 29|        IT|
+------+--------+------+---+----------+



#### Task 8: Add 10% Bonus to Salaries

Perform a transformation to add a new column named "SalaryAfterBonus" to the DataFrame. Calculate the new salary by adding a 10% bonus to each employee's salary.


In [25]:
from pyspark.sql.functions import col, round

# Add a new column "SalaryAfterBonus" with 10% bonus added to the original salary
employees_df = employees_df.withColumn("SalaryAfterBonus", round(col("Salary") * 1.1, 3))
employees_df.show()

+------+---------+------+---+----------+----------------+
|Emp_No| Emp_Name|Salary|Age|Department|SalaryAfterBonus|
+------+---------+------+---+----------+----------------+
|   198|   Donald|  2600| 29|        IT|          2860.0|
|   199|  Douglas|  2600| 34|     Sales|          2860.0|
|   200| Jennifer|  4400| 36| Marketing|          4840.0|
|   201|  Michael| 13000| 32|        IT|         14300.0|
|   202|      Pat|  6000| 39|        HR|          6600.0|
|   203|    Susan|  6500| 36| Marketing|          7150.0|
|   204|  Hermann| 10000| 29|   Finance|         11000.0|
|   205|  Shelley| 12008| 33|   Finance|         13208.8|
|   206|  William|  8300| 37|        IT|          9130.0|
|   100|   Steven| 24000| 39|        IT|         26400.0|
|   101|    Neena| 17000| 27|     Sales|         18700.0|
|   102|      Lex| 17000| 37| Marketing|         18700.0|
|   103|Alexander|  9000| 39| Marketing|          9900.0|
|   104|    Bruce|  6000| 38|        IT|          6600.0|
|   105|    Da

#### Task 9: Find Maximum Salary by Age

Group the data by age and calculate the maximum salary for each age group. Display the result.


In [28]:
from pyspark.sql.functions import max

# Group data by age and calculate the maximum salary for each age group
result = employees_df.groupBy("Age").agg(
    max("Salary").alias("MaxSalary")
)
result.show()

+---+---------+
|Age|MaxSalary|
+---+---------+
| 31|     8200|
| 34|     7800|
| 28|    12008|
| 27|    17000|
| 26|     3600|
| 37|    17000|
| 35|     9000|
| 39|    24000|
| 38|     6000|
| 29|    10000|
| 32|    13000|
| 33|    12008|
| 30|     8000|
| 36|     7900|
+---+---------+



#### Task 10: Self-Join on Employee Data

Join the "employees_df" DataFrame with itself based on the "Emp_No" column. Display the result.


In [29]:
# Join the DataFrame with itself based on the "Emp_No" column
result = employees_df.join(employees_df, on = "Emp_No", how="inner")
result.show()

+------+---------+------+---+----------+----------------+---------+------+---+----------+----------------+
|Emp_No| Emp_Name|Salary|Age|Department|SalaryAfterBonus| Emp_Name|Salary|Age|Department|SalaryAfterBonus|
+------+---------+------+---+----------+----------------+---------+------+---+----------+----------------+
|   198|   Donald|  2600| 29|        IT|          2860.0|   Donald|  2600| 29|        IT|          2860.0|
|   199|  Douglas|  2600| 34|     Sales|          2860.0|  Douglas|  2600| 34|     Sales|          2860.0|
|   200| Jennifer|  4400| 36| Marketing|          4840.0| Jennifer|  4400| 36| Marketing|          4840.0|
|   201|  Michael| 13000| 32|        IT|         14300.0|  Michael| 13000| 32|        IT|         14300.0|
|   202|      Pat|  6000| 39|        HR|          6600.0|      Pat|  6000| 39|        HR|          6600.0|
|   203|    Susan|  6500| 36| Marketing|          7150.0|    Susan|  6500| 36| Marketing|          7150.0|
|   204|  Hermann| 10000| 29|   Finan

#### Task 11: Calculate Average Employee Age

Calculate the average age of employees using the built-in aggregation function. Display the result.


In [30]:
# Calculate the average age of employees
from pyspark.sql.functions import avg 

result = employees_df.agg(
    avg("Age").alias("AvgAge")
)
result.show()

+------+
|AvgAge|
+------+
| 33.56|
+------+



#### Task 12: Calculate Total Salary by Department

Calculate the total salary for each department using the built-in aggregation function. Display the result.


In [31]:
# Calculate the total salary for each department. Hint - User GroupBy and Aggregate functions
from pyspark.sql.functions import sum 
result = employees_df.groupBy("Department").agg(
    sum("Salary").alias("TotalSalary")
)
result.show()

+----------+-----------+
|Department|TotalSalary|
+----------+-----------+
|     Sales|      71408|
|        HR|      46700|
|   Finance|      57308|
| Marketing|      59700|
|        IT|      74000|
+----------+-----------+



#### Task 13: Sort Data by Age and Salary

Apply a transformation to sort the DataFrame by age in ascending order and then by salary in descending order. Display the sorted DataFrame.


In [32]:
# Sort the DataFrame by age in ascending order and then by salary in descending order
sorted_df = employees_df.orderBy(col("Age").asc(), col("Salary").desc())
sorted_df.show()

+------+---------+------+---+----------+----------------+
|Emp_No| Emp_Name|Salary|Age|Department|SalaryAfterBonus|
+------+---------+------+---+----------+----------------+
|   137|   Renske|  3600| 26| Marketing|          3960.0|
|   101|    Neena| 17000| 27|     Sales|         18700.0|
|   114|      Den| 11000| 27|   Finance|         12100.0|
|   108|    Nancy| 12008| 28|     Sales|         13208.8|
|   130|    Mozhe|  2800| 28| Marketing|          3080.0|
|   126|    Irene|  2700| 28|        HR|          2970.0|
|   204|  Hermann| 10000| 29|   Finance|         11000.0|
|   115|Alexander|  3100| 29|   Finance|          3410.0|
|   134|  Michael|  2900| 29|     Sales|          3190.0|
|   198|   Donald|  2600| 29|        IT|          2860.0|
|   140|   Joshua|  2500| 29|   Finance|          2750.0|
|   136|    Hazel|  2200| 29|        IT|          2420.0|
|   120|  Matthew|  8000| 30|        HR|          8800.0|
|   110|     John|  8200| 31| Marketing|          9020.0|
|   127|    Ja

#### Task 14: Count Employees in Each Department

Calculate the number of employees in each department. Display the result.


In [35]:
from pyspark.sql.functions import count

# Calculate the number of employees in each department
result = employees_df.groupBy("Department").agg(
    count("Emp_No").alias("NumberOfEmp")
)
result.show()

+----------+-----------+
|Department|NumberOfEmp|
+----------+-----------+
|     Sales|         13|
|        HR|          8|
|   Finance|         10|
| Marketing|          9|
|        IT|         10|
+----------+-----------+



#### Task 15: Filter Employees with the letter o in the Name

Apply a filter to select records where the employee's name contains the letter `'o'`. Display the filtered DataFrame.


In [37]:
# Apply a filter to select records where the employee's name contains the letter 'o'
filtered_df = employees_df.filter(col("Name").like("%o%"))
filtered_df.show()

+------+-----------+------+---+----------+----------------+
|Emp_No|   Emp_Name|Salary|Age|Department|SalaryAfterBonus|
+------+-----------+------+---+----------+----------------+
|   198|     Donald|  2600| 29|        IT|          2860.0|
|   199|    Douglas|  2600| 34|     Sales|          2860.0|
|   110|       John|  8200| 31| Marketing|          9020.0|
|   112|Jose Manuel|  7800| 34|        HR|          8580.0|
|   130|      Mozhe|  2800| 28| Marketing|          3080.0|
|   133|      Jason|  3300| 38|     Sales|          3630.0|
|   139|       John|  2700| 36|     Sales|          2970.0|
|   140|     Joshua|  2500| 29|   Finance|          2750.0|
+------+-----------+------+---+----------+----------------+



# Congratulations! You have completed the project.

Now you know how to create a DataFrame from a CSV data file and perform a variety of DataFrame transformations and actions using Spark SQL.


## Authors


Raghul Ramesh


Lavanya T S


<!--## Change Log -->


<!--|Date (YYYY-MM-DD)|Version|Changed By|Change Description|
|-|-|-|-|
|2023-09-01|0.1|Lavanya T S|Initial version|
|2023-09-11|0.2|Pornima More|QA pass with edits|-->


<h3 align="center"> &#169; IBM Corporation. All rights reserved. <h3/>
